In [7]:
import torch
import torch.nn as nn

class SentimentLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()

        self.lstm = nn.LSTM(
            input_size=input_dim,      # ✅ correct name
            hidden_size=hidden_dim,     # ✅ correct name
            num_layers=num_layers,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]   # last timestep
        return self.fc(out)



In [11]:
import joblib
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [12]:
# LSTM
model = SentimentLSTM(input_dim=10, hidden_dim=128, num_layers=2)
model.load_state_dict(torch.load("lstm_model.pth"))
model.eval()

scaler_X = joblib.load("scaler_X.pkl")
scaler_y = joblib.load("scaler_y.pkl")

# FinBERT
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
sent_model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
sent_model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [39]:
import requests
import datetime

NEWS_API_KEY = "571ad5b98b254679854e5103fa3caecf"
QUERY = "Reliance Industries"

today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

url = (
    f"https://newsapi.org/v2/everything?"
    f"q={QUERY}&"
    f"from={yesterday}&to={today}&"
    f"language=en&"
    f"sortBy=relevancy&"
    f"apiKey={NEWS_API_KEY}"
)

response = requests.get(url).json()

articles = response.get("articles", [])


In [43]:
import torch
import numpy as np

def sentiment_one_news(text, max_tokens=512):
    if text.strip() == "":
        return 0

    inp = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        add_special_tokens=False
    )

    input_ids = inp["input_ids"][0]
    scores = []

    for i in range(0, len(input_ids), max_tokens):
        chunk_ids = input_ids[i:i + max_tokens]

        inputs = {
            "input_ids": chunk_ids.unsqueeze(0),
            "attention_mask": torch.ones_like(chunk_ids).unsqueeze(0)
        }

        with torch.no_grad():
            outputs = sent_model(**inputs)
            label = torch.argmax(outputs.logits, dim=1).item()

        # LABEL → SCORE (NO PROBABILITIES)
        if label == 2:
            scores.append(1)     # positive
        elif label == 0:
            scores.append(-1)    # negative
        else:
            scores.append(0)     # neutral

    # average sentiment for THIS news
    return sum(scores) / len(scores)


In [44]:
today_text = " ".join(
    article["title"] + ". " + (article["description"] or "")
    for article in articles
)
today_text

"Solid-State Battery or Supercapacitor, Donut Lab’s Claims Don’t Add Up. What if the future of energy storage could be redefined in just five minutes? That’s the bold promise behind the world’s first solid-state battery unveiled by Donut Lab, a breakthrough that has sparked both excitement and skepticism across industries. Ziroth … Q3 Results Today: GAIL, Delhivery Among Over 75 Companies To Declare Earnings. Sun Pharmaceutical Industries, Birla Corporation, Moschip Technologies and RR Kabel are among the companies that will declare earnings on Jan. 31. Q3 Results LIVE Updates: Sun Pharma, CDSL, IDFC First Bank In Focus. Q3 Results Today LIVE Updates: Catch all the live updates, earnings numbers and developments from companies reporting Q3FY26 results here through the day. US pitches Venezuelan crude to India as its Russian oil imports slow, sources say. India plans to reduce Russian crude oil imports by several hundred thousand barrels per day following US tariff increases on such pur

In [45]:
today_sentiment = sentiment_one_news(today_text)
print("Today's sentiment:", today_sentiment)
  # sanity check


Today's sentiment: -1.0


In [46]:
import pandas as pd
import numpy as np

# -----------------------------
# RAW DATA (what you gave)
# -----------------------------

data = [

    ("2026-01-01",1575.599976,1592.5,1571.099976,1573.699951,6408128),
    ("2026-01-02",1592.300049,1594.599976,1578.199951,1580.0,6602230),

    ("2026-01-05",1578.099976,1611.800049,1575.300049,1593.0,8704973),
    ("2026-01-06",1507.599976,1569.0,1496.300049,1569.0,27417125),
    ("2026-01-07",1504.199951,1520.0,1498.0,1510.0,11199340),
    ("2026-01-08",1470.599976,1503.900024,1468.800049,1500.0,16518684),
    ("2026-01-09",1475.300049,1479.900024,1465.0,1465.0,8335311),
    ("2026-01-12",1483.199951,1485.300049,1451.0,1475.300049,8883745),
    ("2026-01-13",1452.800049,1485.800049,1444.699951,1485.0,13499760),
    ("2026-01-14",1458.800049,1467.0,1440.199951,1444.0,8321764),
    ("2026-01-15",1458.800049,1458.800049,1458.800049,1458.800049,0),
    ("2026-01-16",1457.900024,1480.0,1455.099976,1458.800049,17167161),
    ("2026-01-19",1413.599976,1450.599976,1403.300049,1450.599976,20392765),
    ("2026-01-20",1394.0,1416.0,1390.0,1410.699951,13189498),
    ("2026-01-21",1404.599976,1412.900024,1373.0,1389.5,17352720),
    ("2026-01-22",1402.5,1419.5,1395.0,1410.099976,15721693),
    ("2026-01-23",1386.099976,1407.0,1381.0,1401.800049,9837802),
    ("2026-01-27",1380.5,1391.599976,1368.0,1380.0,26302396),
    ("2026-01-28",1396.699951,1409.900024,1384.900024,1385.0,11121657),
    ("2026-01-29",1391.0,1402.900024,1384.0,1402.900024,20846683),
    ("2026-01-30",1347.0,1411.0,1335.0,1396.0,9714528)
]



df = pd.DataFrame(
    data, columns=["Date","Close","High","Low","Open","Volume"]
)
df["Date"] = pd.to_datetime(df["Date"])

# -----------------------------
# FEATURES
# -----------------------------
df["log_return"] = np.log(df["Close"] / df["Close"].shift(1))

ema5 = df["Close"].ewm(span=5, adjust=False).mean()
ema20 = df["Close"].ewm(span=20, adjust=False).mean()
df["macd"] = ema5 - ema20

delta = df["Close"].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
rs = gain.rolling(14).mean() / loss.rolling(14).mean()
df["rsi_14"] = 100 - (100 / (1 + rs))

df["volatility_20"] = df["log_return"].rolling(20).std()

# -----------------------------
# TODAY'S FEATURE VECTOR
# -----------------------------
today = df.dropna().iloc[-1]

X_today = np.array([
    today["log_return"],
    today["Open"],

    today['High'],
    today['Low'],
    today["Close"],
    today["Volume"],
    today_sentiment,
    today["rsi_14"],
    today["macd"],
    today["volatility_20"]
])

print("X_today values:")
print(X_today)
print("Shape:", X_today.shape)


X_today values:
[-3.21430155e-02  1.39600000e+03  1.41100000e+03  1.33500000e+03
  1.34700000e+03  9.71452800e+06 -1.00000000e+00  1.94086246e+01
 -5.95607578e+01  1.55586862e-02]
Shape: (10,)


In [13]:
# last SEQ_LEN rows = today’s sequence
# X_today = df_final[features].values[-SEQ_LEN:]
SEQ_LEN = 20
INPUT_DIM = 10
assert X_seq.shape == (SEQ_LEN, INPUT_DIM)
X_seq_scaled = scaler_X.transform(X_seq)

X_today_t = torch.tensor(X_seq_scaled).unsqueeze(0).float()



assert X_today_t.shape == (1, SEQ_LEN, INPUT_DIM)
X_today_scaled = scaler_X.transform(X_today)
X_today_t = torch.tensor(X_today_scaled).unsqueeze(0).float()


with torch.no_grad():
    pred_scaled = model(X_today_t).numpy().flatten()

pred_return = scaler_y.inverse_transform(
    pred_scaled.reshape(-1, 1)
)[0][0]

print(pred_return)

NameError: name 'df_final' is not defined

In [ ]:
with torch.no_grad():
    pred_scaled = model(X_today_t).numpy().flatten()

pred_return = scaler_y.inverse_transform(pred_scaled.reshape(-1,1))[0][0]


In [ ]:
today_text = df_news["Headline"].iloc[-1]
today_sentiment = average_sentiment(today_text)   # -1 / 0 / +1


In [ ]:
signal = get_trade_signal(
    lstm_prediction=pred_return,
    sentiment_score=today_sentiment,
    threshold=0.001
)

print("Predicted return:", pred_return)
print("Sentiment:", today_sentiment)
print("TRADE SIGNAL:", signal)
